In [5]:
# pip install -U scikit-learn
import numpy as np
import pandas as pd

In [6]:
from sklearn.metrics import mean_absolute_percentage_error, \
mean_absolute_error, r2_score, mean_squared_error

In [22]:
df = pd.read_pickle('./bitcoin_etl_Max_1d.pk1')
df = df[['Open','High','Low','Close', 'Volume']]
df.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [23]:
df['ClosePrediction'] = df['Close'].shift(1)
df.head(5)

,Open,High,Low,Close,Volume,ClosePrediction
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,457.334015
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,424.440002
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,394.795990
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,408.903992


In [24]:
y_true = df.iloc[1:]['Close']
y_pred = df.iloc[1:]['ClosePrediction']

### Metrics 
Main idea: get a fell for how the values related to one another. what's "good"? What is "bad"? If the $R^2$ is "good", will the MAE also be "good"?

In [25]:
# SSE
(y_true - y_pred).dot(y_true - y_pred)

1381661014.5120156

In [26]:
# MSE
mean_squared_error(y_true, y_pred)

530184.5796285555

In [27]:
# MSE again
(y_true - y_pred).dot(y_true - y_pred) / len(y_true)

530184.5796285555

In [28]:
# RMSE
mean_squared_error(y_true, y_pred, squared = False)

728.1377477020096

In [29]:
# RMES again 
np.sqrt((y_true - y_pred).dot(y_true - y_pred) / len(y_true))

728.1377477020096

In [30]:
# MAE 
mean_absolute_error(y_true, y_pred)

296.1690013472703

In [31]:
# R^2
r2_score(y_true,y_pred)

0.9975288775133944

In [32]:
# MAPE
mean_absolute_percentage_error(y_true, y_pred)

0.025782689278728826

### sMAPE

$E = \frac{1}{N}\sum \limits_{i = 1}^{N}\frac{|y_{i}-\hat{y}_{i}|}{|y_{i} + \hat{y}_{i}|/2} $


In [36]:
def smape (y_true, y_pred):
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true)+ np.abs(y_pred))/2
    ratio = numerator /denominator 
    return ratio.mean()

smape(y_true, y_pred)

0.025723444070844877

***
$\mathbf{\text{Gradient Tree Boosting Algorithm}}$<br>
***
1.&emsp;Initialize model with a constant value $$f_{0}(x) = \textrm{arg min}_{\gamma} \sum \limits _{i=1} ^{N} L(y_{i}, \gamma)$$
2.&emsp;For m = 1 to M:<br>
&emsp;&emsp;(a)&emsp;For $i = 1,2,...,N$ compute<br>
    $$r_{im} = - \displaystyle \Bigg[\frac{\partial L(y_{i}, f(x_{i}))}{\partial f(x_{i})}\Bigg]_{f=f_{m−1}}$$
&emsp;&emsp;(b)&emsp;Fit a regression tree to the targets $r_{im}$ giving terminal regions<br>
&emsp;&emsp;&emsp;&emsp;$R_{jm}, j = 1, 2, . . . , J_{m}.$<br><br>
&emsp;&emsp;(c)&emsp;For $j = 1, 2, . . . , J_{m}$ compute<br>
$$\gamma_{jm} = \underset{\gamma}{\textrm{arg min}} \sum \limits _{x_{i} \in R_{jm}} L(y_{i}, f_{m−1}(x_{i}) + \gamma)$$
<br>
&emsp;&emsp;(d)&emsp;Update $f_{m}(x) = f_{m−1}(x) + \sum _{j=1} ^{J_{m}} \gamma_{jm} I(x \in R_{jm})$<br><br>
3. Output $\hat{f}(x) = f_{M}(x)$
***